In [1]:
#load libraries
from aicsimageio import AICSImage #handles czi data
import numpy as np
from napari.viewer import Viewer

In [2]:
#read in one image stack
img_movie = AICSImage("C:/Users/mrfel/Desktop/Python_random/Movie1.czi")

In [7]:
#create instance of napari viewer
viewer = Viewer()

11-Jun-23 17:49:49 - vispy    - WARNING  - QWindowsWindow::setGeometry: Unable to set geometry 765x783-499+24 (frame: 781x822-507-7) on QWidgetWindow/"_QtMainWindowClassWindow" on "\\.\DISPLAY2". Resulting geometry: 610x625-500+17 (frame: 626x664-508-14) margins: 8, 31, 8, 8 minimum size: 374x573 MINMAXINFO maxSize=0,0 maxpos=0,0 mintrack=390,612 maxtrack=0,0)


In [27]:
#add movie to napari viewer
viewer.add_image(img_movie.data, colormap="gray", name="OriginalMovie")

<Image layer 'OriginalMovie' at 0x1793ab123d0>

In [24]:
#create average and median projection of movie and add to napari
img_median = np.median(img_movie.data, axis = 0)
img_mean = np.mean(img_movie.data, axis = 0)
img_sum = np.sum(img_movie.data, axis = 0)
img_max = np.max(img_movie.data, axis = 0)

In [25]:
viewer.add_image(img_median,colormap="gray",name="Median_Projection")
viewer.add_image(img_mean,colormap="gray",name="Mean_Projection")
viewer.add_image(img_sum,colormap="gray",name="Sum_Projection")
viewer.add_image(img_max,colormap="gray",name="Max_Projection")

<Image layer 'Max_Projection' at 0x17946c166a0>

In [26]:
movie_median = img_movie.data - img_median
movie_mean = img_movie.data - img_mean
movie_sum = img_movie.data - img_sum
movie_max = img_movie.data - img_max

In [28]:
viewer.add_image(movie_median,colormap="gray",name="MedianMovie")
viewer.add_image(movie_mean,colormap="gray",name="MeanMovie")
viewer.add_image(movie_sum,colormap="gray",name="SumMovie")
viewer.add_image(movie_max,colormap="gray",name="MaxMovie")
#Subtraction of sum or max is not a good idea

<Image layer 'MaxMovie' at 0x1793fe931c0>